In [ ]:
import sys
import time
from dataclasses import dataclass
from PIL import Image, ImageDraw

sys.path.append('../utils')
from pyutils import *

In [ ]:
@dataclass
class Edge:
    path: list[Pt]
    dead: bool = False

In [ ]:
def scan_vertices(mat: StrMatrix) -> dict[Pt, dict[Pt, Edge]]:
    verts: dict[Pt, dict[Pt, Edge]] = {}
    for pt, val in mat_iter(mat):
        if val == '#':
            continue
        branches: list[Pt] = []
        for direc in Pt.cardinals():
            future = pt + direc
            if matget(mat, future) != '#':
                branches.append(future)
        if len(branches) > 2:
            verts[pt] = {br:Edge([pt]) for br in branches}

    return verts

In [ ]:
def build_graph(maze: Matrix, start: Pt, end: Pt, gif: list[Image.Image] | None = None) -> dict[Pt, dict[Pt, Edge]]:
    verts: dict[Pt, dict[Pt, Edge]] = scan_vertices(maze)
    if start not in verts:
        verts[start] = {start + br:Edge([start]) for br in Pt.cardinals() if matget(maze, start + br) != '#'}
    if end not in verts:
        verts[end] = {end + br:Edge([end]) for br in Pt.cardinals() if matget(maze, end + br) != '#'}

    def explore_until_vert(head: Pt, edge: Edge):
        cursor: Pt = head
        while True:
            edge.path.append(cursor)
            if cursor in verts:
                return
            branches: list[Pt] = []
            for br in Pt.cardinals():
                future = cursor + br
                if (matget(maze, future) != '#') and (future not in edge.path):
                    branches.append(cursor + br)
            if len(branches) == 0:
                edge.dead = True
                return
            if len(branches) == 1:
                cursor = branches[0]
                continue

    cursor: Pt = start
    seen: set[Pt] = set()
    for k in verts:
        for head, edge in verts[k].items():
            explore_until_vert(head, edge)
            seen.update(edge.path)
            if gif is not None:
                gif.append(matimg(maze, colflt={
                    (lambda p,v: v == '#'): 'black',
                    (lambda p,v: p in seen): 'grey',
                    (lambda p,v: p in edge.path): 'red',
                    (lambda p,v: p in verts): 'blue',
                    (lambda p,v: p == k): 'magenta'
                }))

    return verts

In [ ]:
def score_path(path: list[Pt], startdir: Pt = Pt(0, 1)) -> int:
    score = 0
    if startdir != path[1] - path[0]:
        score += 1000
    for n, _ in enumerate(path):
        if n + 2 >= len(path):
            break
        a = path[n]
        b = path[n + 1]
        c = path[n + 2]
        if a[0] != c[0] and a[1] != c[1]:
            score += 1000
    score += (len(path) - 1)
    return score

In [ ]:
def best_path(graph: dict[Pt, dict[Pt, Edge]], start: Pt, end: Pt, gif: list[Image.Image] | None = None) -> list[Pt]:
    vertstack: list[Pt] = []
    dead_verts: list[Pt] = []

    vert: Pt = start
    score_total: int = 0
    last_dir: Pt = Pt(0, 1)

    count = 0
    while True:
        if gif is not None:
            gif.append(matimg(maze, colflt={
                (lambda p,v: v == '#'): 'black',
                (lambda p,v: p in graph): 'blue',
                (lambda p,v: p in vertstack): 'magenta',
                (lambda p,v: p in dead_verts): 'grey',
                (lambda p,v: p == vert): 'lime'
            }))
        count += 1
        # if count == 50:
        #     print('BREAK!')
        #     break
        edges: dict[Pt, Edge] = {h:e for h, e in graph[vert].items() if (not e.dead) and (e.path[-1] not in vertstack + dead_verts)}
        scores: dict[Pt, int] = {}
        if len(edges) == 0:
            dead_verts.append(vert)
            vert = vertstack.pop()
            continue
        for head, edge in edges.items():
            tail: Pt = edge.path[-1]
            if tail == end:
                vertstack.append(vert)
                vertstack.append(tail)
                return vertstack
            scores[head] = score_path(edge.path, last_dir)
        vertstack.append(vert)
        best = edges[sorted(edges, key=lambda k: scores[k] * k.distance(end))[0]].path
        last_dir = best[-1] - best[-2]
        vert = best[-1]
        score_total += scores[best[1]]

In [ ]:
def follow_verts(mat: Matrix, graph: dict[Pt, dict[Pt, Edge]], verts: list[Pt], gif: list[Image.Image] | None = None) -> list[Pt]:
    full_path: list[Pt] = []
    for n, vt in enumerate(verts):
        if n + 1 == len(verts):
            break
        # Consider optimizing the path by checking if the end vertex of any edge matches
        # *any* upcoming vertex, i.e. in verts[n:], and the edge that leads to the vertex
        # that is closest to the end of the list
        edges: list[Edge] = [
            graph[vt][head] \
            for head in sorted(graph[vt], key=lambda i: len(graph[vt][i].path)) \
            if (not graph[vt][head].dead) and (graph[vt][head].path[-1] == verts[n + 1])
        ]
        full_path.extend(edges[0].path[:-1])
        if gif is not None:
            gif.append(matimg(mat, colflt={
                (lambda p,v: v == '#'): 'black',
                (lambda p,v: p in full_path): 'red',
                (lambda p,v: p in verts): 'blue',
            }))
    full_path.append(verts[-1])
    return full_path

In [ ]:
sample = readutf8('sample.txt')
real = readutf8('input.txt')
maze = strtomat(real)

In [ ]:
start, end = Pt(len(maze) - 2, 1), Pt(1, len(maze[0]) - 2)

In [ ]:
# _frames = []
ta = time.perf_counter()
graph = build_graph(maze, start, end)
tb = time.perf_counter()
print(f'{tb - ta:.8f}')
print(len(graph), f'@ {sys.getsizeof(graph) / 1000}KB')
# _frames[0].save('out_a.gif', save_all=True, append_images=_frames[1:], duration=100)

In [ ]:
# _frames = []
ta = time.perf_counter()
retpath = best_path(graph, start, end)
tb = time.perf_counter()
print(f'{tb - ta:.8f}')
print(len(retpath), f'@ {sys.getsizeof(retpath) / 1000}KB')
# _frames[0].save('out_b.gif', save_all=True, append_images=_frames[1:], duration=100)

In [ ]:
_finalframes = []
ta = time.perf_counter()
final_path = follow_verts(maze, graph, retpath)
tb = time.perf_counter()
print(f'{tb - ta:.8f}')
print(len(retpath), f'@ {sys.getsizeof(retpath) / 1000}KB')
# _finalframes[0].save('out_c.gif', save_all=True, append_images=_finalframes[1:], duration=100)

In [ ]:
score_path(final_path)